In [1]:
from openai import OpenAI
import os

In [8]:
client = OpenAI(api_key="")

In [38]:
# Step 1: Create an Assistant
assistant = client.beta.assistants.create(
    name="Document Parser",
    instructions="You are an assistant to parser web and show details asked.",
    model="gpt-3.5-turbo",
    tools=[{"type": "file_search"}]
)

In [29]:
assistant

Assistant(id='asst_d5Ub9Zyvlg3FYuyF6j2TZzpI', created_at=1717603343, description=None, instructions='You are an assistant to parser web and show details asked.', metadata={}, model='gpt-3.5-turbo', name='Web Parser', object='assistant', tools=[FileSearchTool(type='file_search', file_search=None)], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=[])), top_p=1.0)

In [39]:
vector_store = client.beta.vector_stores.create(name="Course Contents")

# Ready the files for upload to OpenAI
file_paths = ["Welcome to our extensive MLOps Bootcamp.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
file_streams

[<_io.BufferedReader name='Welcome to our extensive MLOps Bootcamp.pdf'>]

In [40]:
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [46]:
vector_store.id

'vs_BrlqHc9YnLA0OvIxRk0vS13h'

In [41]:
# Step 3: Update the assistant to use the new Vector Store
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [42]:
# Step 4: Create a thread
# Upload the user provided file to OpenAI
message_file = client.files.create(
    file=open("Welcome to our extensive MLOps Bootcamp.pdf", "rb"), purpose="assistants"
)
message_file

FileObject(id='file-zvyIhX69z4I9op9C35v2FCP3', bytes=350109, created_at=1717603471, filename='Welcome to our extensive MLOps Bootcamp.pdf', object='file', purpose='assistants', status='processed', status_details=None)

In [43]:
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Display course contents",
            # Attach the new file to the message.
            "attachments": [
                {"file_id": message_file.id, "tools": [{"type": "file_search"}]}
            ],
        }
    ]
)
# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

ToolResourcesFileSearch(vector_store_ids=['vs_2GzRQNtaJmh9b92flCm0wDNL'])


In [44]:
# Use the create and poll SDK helper to create a run and poll the status of
# the run until it's in a terminal state.

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))


In [47]:
run

Run(id='run_6vbyCw2m5ZiXJ3nlmNFeNFsf', assistant_id='asst_oPTOTJvfOc5As77zdD2uXB02', cancelled_at=None, completed_at=None, created_at=1717603478, expires_at=None, failed_at=1717603483, incomplete_details=None, instructions='You are an assistant to parser web and show details asked.', last_error=LastError(code='rate_limit_exceeded', message='You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.'), max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', required_action=None, response_format='auto', started_at=1717603478, status='failed', thread_id='thread_2yd643MOsRrKVSzc03O6ea8E', tool_choice='auto', tools=[FileSearchTool(type='file_search', file_search=None)], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=0, prompt_tokens=0, total_tokens=0), 

In [45]:
messages

[]

In [37]:
message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

IndexError: list index out of range